<a href="https://colab.research.google.com/github/ncsu-geoforall-lab/GIS582-assignments/blob/main/3AB%20-%20Analysis/3B_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 3B: Buffers, cost surfaces, least cost path

**Course:** [GIS 582 - Geospatial Modeling and Analysis](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/index.html)  
**Institution:** [NC State University, Center for Geospatial Analytics](https://cnr.ncsu.edu/geospatial/)
**Instructors:** Helena Mitasova, Corey White, and team

## Learning Objectives

In this tutorial, you will learn how to:
- measuring distance, proximity operators
- point, line, and area buffers
- cost surfaces, least cost path

## Tutorial Outline

1. Environment Setup
2. Buffers
3. Cost Surfaces
4. Optional

---
## Part 1: Environment Setup

### Install GRASS

**Important:** This setup takes 3-5 minutes. You'll need to run it each time you start a new Colab session.

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev

Check that GRASS is installed by asking which version is there.

In [ ]:
!grass --version

Check which Python version is running.

In [ ]:
import sys

v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

In [ ]:
import subprocess
import os
from pathlib import Path

# Ask GRASS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

In [ ]:
import grass.script as gs
import grass.jupyter as gj

### Download North Carolina Sample Dataset

This dataset includes elevation, land cover, roads, streams, and more.

In [ ]:
!grass --tmp-project XY --exec g.download.project url=https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.tar.gz path=/content

### Initialize GRASS Session

In [ ]:
# Start GRASS session
grassdata = "/content"
location = "nc_spm_08_grass7"
mapset = "user1"  # Create a new mapset for our work

# Start GRASS Session
session = gj.init(Path(location, mapset))

# Set computational region
gs.run_command('g.region', raster='elevation@PERMANENT', flags='p')

---
## Part 2: Buffers

### 2.1 - Find developed areas potentially impacted by noise from highways.

Set region and create buffers along major roads.

In [ ]:
!g.region raster=landuse96_28m -p
!v.to.rast roadsmajor out=roadsmajor_28m use=value
!r.buffer roadsmajor_28m output=roads_buffers distances=250,500,2500

Intersect developed areas from landuse map with road buffers.

In [ ]:
!r.mapcalc "noise = if(landuse96_28m==1 || landuse96_28m==2, roads_buffers, null())"
!r.colors noise color=ryg

Transfer the category labels and compute the affected area.

Contents of [`noise_cats.txt`](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/noise_cats.txt):

```text
1:source
2:high
3:moderate
4:low
```

In [ ]:
!printf '%s\n' "1:source" "2:high" "3:moderate" "4:low" | r.category noise rules=- separator=:
!r.report -n noise units=p,h

Display the results.

In [ ]:
noise_map = gj.Map(filename="noise.png")
noise_map.d_rast(map="noise")
noise_map.d_vect(map="streets_wake", color="grey")
noise_map.d_legend(raster="noise", at=[5, 35, 5, 8], flags="cb")
noise_map.show()

#### Question 2.1

**What is the total developed area iin [ha] within the 250m from the major roads?**

`Add awnsers here.`

### 2.2 - Find schools potentially affected by high levels of noise

Convert the schools vector to raster using `CORECAPACI` attribute (school capacity).

In [ ]:
!v.to.rast schools_wake output=schoolscap_10m use=attr attrcolumn=CORECAPACI type=point

Use map algebra to overlay with noise impact buffers and compute the number of students exposed to noise (see result of [r.univar](https://grass.osgeo.org/grass76/manuals/r.univar.html)).

In [ ]:
!r.mapcalc "schools_noise = if(int(schoolscap_10m) && roads_buffers == 2, int(schoolscap_10m), null())"
!r.to.vect schools_noise output=schools_noise type=point
!r.univar schools_noise

Display the results.

In [ ]:
schools_map = gj.Map(filename="mynoisemap.png")
schools_map.d_vect(map="schools_wake", icon="basic/circle", size=10, fill_color="blue", legend_label="schools")
schools_map.d_vect(map="schools_noise", icon="basic/circle", size=14, fill_color="cyan", color="black", legend_label="noise risk schools")
schools_map.d_legend_vect(at=[70, 15])
schools_map.show()


#### Question 2.2

**How many students are potentially affected by noise based on the school capacities and less than 250m distance to major roads?**

`Add awnsers here.`

---
## Part 3: Cost surfaces

Neighborhood operations analyze each cell based on its surrounding cells within a defined neighborhood (moving window).

### 3.1 - Compute cumulative cost surface to a given accident site based on speed limits

Set region to Wake County at 30m resolution.

In [ ]:
!g.region swwake_30m -p

View the categories of the streets_wake vector layer to see speed limit attribute.

In [ ]:
!v.info -c streets_wake

Create a raster form `street_wake` using the `SPEED` attribute. Use 5mi/hr speed limit for off-road areas (nulls).

In [ ]:
!v.to.rast streets_wake output=streets_speedtmp use=attr attrcolumn=SPEED type=line
!r.mapcalc "streets_speed = if(isnull(streets_speedtmp),5,streets_speedtmp)"
!r.info streets_speed

Display the results.

In [ ]:
gs.run_command('r.colors', map='streets_speed', color='inferno', flags='n',)
speed_map = gj.Map(filename="myspeedmap.png")
speed_map.d_rast(map="streets_speed")
speed_map.d_legend(raster="streets_speed", at=[5,40,2,5], use=[5,25,35,45,65])
speed_map.show()

Import the accident point location from GeoJSON file [fire_pt.json](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/fire_pt.json).

In [ ]:
!v.in.ogr type=point key=cat input=https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/fire_pt.json output=fire_pt

Assign travel time to cross a 30m grid cell in hours.

> Note that cost in GRASS is defined as travel time per cell.

Compute cumulative cost surface to the given point.

In [ ]:
!r.mapcalc "streets_travtime = 0.018641/streets_speed"
!r.cost -k streets_travtime output=streets_cost start_points=fire_pt

#### Question 3.1

**Where does the `0.018641` constant come from? You can modify the expression to get time in minutes.**

`Add awnsers here.`

Compute isochrones and display the cumulative cost surface map.

In [ ]:
gs.run_command('r.contour',
               input='streets_cost',
               output='streets_cost_04',
               step=0.04,
               cut=100,
               overwrite=True)

Display the results.

In [ ]:
gs.run_command("r.colors", map="streets_cost", color="elevation")
cumulative_map = gj.Map(filename="mycumulativecostmap.png")
cumulative_map.d_rast(map="streets_cost")
cumulative_map.d_vect(map="fire_pt", color="red", icon="basic/marker", size=20)
cumulative_map.d_vect(map="streets_cost_04")
cumulative_map.d_legend(raster="streets_cost", at=[5, 50, 2, 5])
cumulative_map.show()

### 3.2 - Find cost (travel time) from selected firestations

First make your own copy of the firestations map and list attributes.

In [ ]:
!g.copy vector=firestations,myfirestations
!v.info -c myfirestations

Then query the cumulative cost surface at the firestations location.

> The travel time in hours will be stored in the attribute column `CVLAG`.

In [ ]:
!v.what.rast myfirestations raster=streets_cost column=CVLAG

View the firestations map.

In [ ]:
firestations_map = gj.Map()
firestations_map.d_rast(map="streets_cost")
firestations_map.d_vect(map="myfirestations", color="red", icon="basic/circle", size=15)
firestations_map.show()

View the attribute table of the firestations map to see travel times and order them and find the lowest cost (shortest time) > 0 (firestations with 0 cost are outside the region).

To do this we will use [v.db.select](https://grass.osgeo.org/grass-stable/manuals/v.db.select.html) to get the attributes of interest in a `JSON` and then display them in a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [ ]:
try:
    import pandas as pd
except ImportError:
    print("Pandas not found. Installing...")
    !pip install pandas
    import pandas as pd

fire_attr_json = gs.parse_command("v.db.select", map="myfirestations", format="json")
fire_records = fire_attr_json["records"]
df_firestations = pd.DataFrame(fire_records)
df_firestations.sort_values(by="CVLAG", inplace=True, ascending=False)
df_firestations.head(10)


Export firestations with traveltime less than 0.1 hr:

In [ ]:
!v.out.ascii input=myfirestations separator=space precision=3 columns=ID,LOCATION,CVLAG where="CVLAG<0.1 AND CVLAG>0"

You should get something like the following:

```text
635775.565 228121.693 19 19 4021 District Dr 0.076
635940.262 225912.796 20 0 5001 Western Blvd 0.037
637386.831 222569.152 21 0 1721 Trailwood Dr 0.071
633178.155 221353.037 52 27 6000 Holly Springs Rd 0.060
```

To get the computed time, you can also query the cumulative cost raster directly using coordinates (in this example it's Western Blvd firestation).

In [ ]:
!r.what map=streets_cost coordinates=635940.262,225912.796 separator=space

Find the least cost path for the two closest stations:

In [ ]:
!r.drain -n input=streets_cost output=route_20Westernb start_coordinates=635940.3,225912.8
!r.drain -n input=streets_cost output=route_52Hollyb start_coordinates=633178.2,221353.0

Display the results.

In [ ]:
gs.run_command('r.colors', map='route_20Westernb', color='ramp')
gs.run_command('r.colors', map='route_52Hollyb', color='ramp')

mylcpmap = gj.Map(filename="mylcpmap.png")
mylcpmap.d_rast(map="streets_cost")
mylcpmap.d_vect(map="fire_pt", color="red", icon="basic/marker", size=20)
mylcpmap.d_vect(map="myfirestations", icon="basic/circle", size=15, fill_color="red")
mylcpmap.d_rast(map="route_20Westernb")
mylcpmap.d_rast(map="route_52Hollyb")
mylcpmap.show()

Print the length of the path in cells (multiply by 30m to get approx. m). You should have the time in hours already from the cost map.

In [ ]:
print("Route 20 Western Blvd")
!r.describe route_20Westernb

In [ ]:
print("Route 52 Holly Blvd")
!r.describe route_52Hollyb

#### Question 3.2

**At what average speed [km/hr] needs the truck travel to get there in estimated time? Is the time, speed and distance realistic?**

`Add awnsers here.`

#### Task 3.2

Do it yourself!

**Find least cost path between the Jordan hall and the Johnson lake dam.** 

Use previously created `streets_travtime` as your input cost.**

Provide the workflow and two maps:

1. Map showing the cumulative cost to get to the Johnson lake dam (start_coordinates=636333,223465)
2. Map showing the resulting least cost path from Jordan Hall (start_coordinates=638875,225450)

converted to a vector line shown in red.

For both maps include `streets_wake` for context. 

**What is the least cost path distance in kilometers between the two locations and how much time in minutes is needed to get to the target destination?** 

> Hint: You don’t need to import the coordinates of the two sites, as they can be provided as input parameters.**

In [ ]:
# Add task code here

In [ ]:
# Add maps displaying the cumulative cost and least cost path here

`Add answers to question here.`

### Part 3.3 - Compute accessibility map for help in search for lost person

Given a point where a lost person was last located we compute a map that represents how far the person could have walked within the surrounding environment. The environmental variables for accessibility are derived from elevation, land use and streets using the equation in the [r.walk](https://grass.osgeo.org/grass-stable/manuals/r.walk.html) manual page and the [friction_rules.txt](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/friction_rules.txt) file.

First, set the computational region and display land cover classes:

In [ ]:
!g.region swwake_30m -p
!r.category landclass96

Recode the landuse map to friction map using the rules in [friction_rules.txt](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/friction_rules.txt).

```text
1:3:0.1:0.1
4:5:10.:10.
6:6:1000.0:1000.0
7:7:0.3:0.3
```

In [ ]:
!printf '%s\n' "1:3:0.1:0.1" "4:5:10.:10." "6:6:1000.0:1000.0" "7:7:0.3:0.3" | r.recode landclass96 out=friction rules=-

Add the impact of streets to friction map - they are missed by the landuse map.

In [ ]:
!r.mapcalc "friction2 = if(streets_speed > 6, 0.1, friction)"

Change the color table of `friction2` to use [friction_color.txt](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/friction_color.txt):

```text
0.1 grey
0.3 brown
10 green
1000 aqua
```

In [ ]:
!printf '%s\n' "0.1 grey" "0.3 brown" "10 green" "1000 aqua" | r.colors friction2 rules=-

Display the friction2 map:

In [ ]:
friction_map = gj.Map(filename="myfrictionmap.png")
friction_map.d_rast(map="friction2")
friction_map.d_legend(raster="friction2", at=[5, 50, 2, 5])
friction_map.show()

Compute the cumulative cost map from the given point and generate isochrones:

In [ ]:
!r.walk -k elevation=elev_ned_30m friction=friction2 output=walkcost start_coordinates=635576,216485 lambda=0.5 max_cost=10000
!r.contour walkcost output=walkcost step=1000 cut=100

To display the input, import coordinates of the point where the lost person was last seen given in the file [lostperson.txt](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/lostperson.txt).

In [ ]:
!printf '%s\n' "635576,216485" | v.in.ascii input=- output=lostperson separator=comma

Display the results.

In [ ]:
gs.run_command("r.colors", map="walkcost", color="viridis", flags="e")

walkcost_map = gj.Map(filename="mywalkcostmap.png")
walkcost_map.d_rast(map="lakes")
walkcost_map.d_rast(map="walkcost")
walkcost_map.d_vect(map="streets_wake")
walkcost_map.d_vect(map="walkcost", color="red")
walkcost_map.d_vect(map="walkcost", color="red", where="level = 6000", width=3)
walkcost_map.d_vect(map="lostperson", color="yellow", fill_color="yellow", icon="basic/marker", size=30)
walkcost_map.d_legend(raster="walkcost", at=[5, 50, 2, 5])
walkcost_map.show()

#### Question 3.3

**How did we ensure that the person does not walk over the lakes?**

`Add answers to question here.`

---
## Part 4: Optional

### 4.1: Find developed areas close to lakes

Set region, create buffers:

In [ ]:
!g.region swwake_30m -p
!r.buffer lakes output=lakes_buff distances=60,120,240,500

In [ ]:
lakes_buff_map = gj.Map()
lakes_buff_map.d_rast(map="lakes_buff")
lakes_buff_map.show()

List categories in land use map to identify category numbers for developed areas. Then run [r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html) to extract the developed areas within the buffers and use [r.support](https://grass.osgeo.org/grass-stable/manuals/r.support.html) to assign the labels from the original buffer raster to the new developed buffer raster map (needed for legend):

In [ ]:
!r.category landuse96_28m
!r.mapcalc "developed_lake = if(landuse96_28m==1 || landuse96_28m==2, lakes_buff, null())"
!r.support developed_lake raster=lakes_buff
!r.category developed_lake

Display the results.

In [ ]:
developed_lake_map = gj.Map(filename="mylakesbuffmap.png")
developed_lake_map.d_rast(map="developed_lake")
developed_lake_map.d_vect(map="streets_wake", color="grey")
developed_lake_map.d_rast(map="lakes")
developed_lake_map.d_legend(raster="developed_lake", at=[5,25,2,5], use=[2,3,4,5])
developed_lake_map.show()

Find the total area within buffers and the developed area in ha:

In [ ]:
!r.report -n lakes_buff units=h
!r.report -n developed_lake units=h

### 4.2: Compute the shortest distance map and cost surface to highways

Set region, convert vector road map to raster.

In [ ]:
!g.region swwake_30m -p
!v.to.rast roadsmajor output=roadsmajor use=val type=line

In [ ]:
!r.mapcalc "area_one = 1"
!r.cost input=area_one output=dist_toroad start_rast=roadsmajor
!r.mapcalc "dist_meters = dist_toroad * (ewres() + nsres())/2."
!r.mapcalc "dist_class = int(dist_meters/500)"

Display the distance to road map.

In [ ]:
dist_roadsmaj_map = gj.Map(filename="dist_roadsmaj.png")
dist_roadsmaj_map.d_rast(map="dist_class")
dist_roadsmaj_map.d_vect(map="roadsmajor", color="black")
dist_roadsmaj_map.show()

Compute the cost surface to major roads based on travel time.

In [ ]:
!r.cost -k input=streets_travtime output=cdist_toroadk start_rast=roadsmajor

Display the results.

In [ ]:
gs.run_command('r.colors', map='cdist_toroadk', color='bgyr')

cost_roadsmaj_map = gj.Map(filename="cost_roadsmaj.png")
cost_roadsmaj_map.d_rast(map="cdist_toroadk")
cost_roadsmaj_map.d_vect(map="streets_wake", color="black")
cost_roadsmaj_map.d_vect(map="roadsmajor", color="red")
cost_roadsmaj_map.d_legend(raster="cdist_toroadk", at=[5, 50, 2, 5], flags="b")
cost_roadsmaj_map.show()